In [1]:
import torch
import matplotlib.pyplot as plt
import sys
from xno.models import XNO
from xno.data.datasets import load_navier_stokes_pt
from xno.utils import count_model_params
from xno.training import AdamW
from xno.training.incremental import IncrementalFNOTrainer
from xno.data.transforms.data_processors import IncrementalDataProcessor
from xno import LpLoss, H1Loss

In [2]:
train_loader, test_loaders, output_encoder = load_navier_stokes_pt(
    n_train=100,
    batch_size=16,
    test_resolutions=[128],
    n_tests=[100, 50],
    test_batch_sizes=[32, 32],
)

/Users/sina/Documents/GitHub_Local/XNO/xno/data/datasets/pt_dataset.py:93: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(


Loading test db for resolution 128 with 100 samples 


/Users/sina/Documents/GitHub_Local/XNO/xno/data/datasets/pt_dataset.py:172: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(Path(root_dir).joinpath(f"{datase

In [3]:
batch = next(iter(train_loader))
type(train_loader), type(batch), batch['x'].shape, batch['y'].shape

(torch.utils.data.dataloader.DataLoader,
 dict,
 torch.Size([16, 1, 128, 128]),
 torch.Size([16, 1, 128, 128]))

In [4]:
batch = next(iter(test_loaders[128]))
type(test_loaders), type(batch), batch['x'].shape, batch['y'].shape

(dict, dict, torch.Size([32, 1, 128, 128]), torch.Size([32, 1, 128, 128]))

In [5]:
len(train_loader.dataset)

100

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
incremental = True
if incremental:
    starting_modes = (2, 2)
else:
    starting_modes = (16, 16)

In [8]:
model = XNO(
    max_n_modes=(16, 16),
    n_modes=(16, 16),
    hidden_channels=32,
    in_channels=1,
    out_channels=1,
    transformation="hno",
    # transformation_kwargs={"wavelet_level": 3, "wavelet_size": [16, 16]}
)
model = model.to(device)
n_params = count_model_params(model)

======== Selected Kernel Description =======
Dimentionality: 2D
Transformation: [ Hilbert Neural Operator (HNO) Kernel ]
>>> Overview:
The HNO applies Hilbert Transform, emphasizing the phase-shifted features of the input
signal for enhanced data representation.

>>> Key Features:
- Focuses on phase information, useful in signal processing.
- Suitable for scenarios requiring advanced spectral analysis.

>>> Reference:
This is an experimental implementation. Currently no formal reference.

================== Config ==================
>>> Normaliztion: None
>>> Activation Function: 



In [9]:
optimizer = AdamW(model.parameters(), lr=8e-3, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=30)

In [10]:
data_transform = IncrementalDataProcessor(
    in_normalizer=None,
    out_normalizer=None,
    device=device,
    subsampling_rates=[2, 1],
    dataset_resolution=16,
    dataset_indices=[2, 3],
    epoch_gap=10,
    verbose=True,
)
data_transform = data_transform.to(device)

Original Incre Res: change index to 0
Original Incre Res: change sub to 2
Original Incre Res: change res to 8


In [11]:
l2loss = LpLoss(d=2, p=2)
h1loss = H1Loss(d=2)
train_loss = h1loss
eval_losses = {"h1": h1loss, "l2": l2loss}
print("\n### N PARAMS ###\n", n_params)
print("\n### OPTIMIZER ###\n", optimizer)
print("\n### SCHEDULER ###\n", scheduler)
print("\n### LOSSES ###")
print("\n### INCREMENTAL RESOLUTION + GRADIENT EXPLAINED ###")
print(f"\n * Train: {train_loss}")
print(f"\n * Test: {eval_losses}")
sys.stdout.flush()


### N PARAMS ###
 2110305

### OPTIMIZER ###
 AdamW (
Parameter Group 0
    betas: (0.9, 0.999)
    correct_bias: True
    eps: 1e-06
    initial_lr: 0.008
    lr: 0.008
    weight_decay: 0.0001
)

### SCHEDULER ###

### LOSSES ###

### INCREMENTAL RESOLUTION + GRADIENT EXPLAINED ###

 * Train: <xno.losses.data_losses.H1Loss object at 0x13bf75690>

 * Test: {'h1': <xno.losses.data_losses.H1Loss object at 0x13bf75690>, 'l2': <xno.losses.data_losses.LpLoss object at 0x13cfa9f00>}


In [12]:
# Finally pass all of these to the Trainer
trainer = IncrementalFNOTrainer(
    model=model,
    n_epochs=10,
    data_processor=data_transform,
    device=device,
    verbose=True,
    incremental_loss_gap=False,
    incremental_grad=True,
    incremental_grad_eps=0.9999,
    incremental_loss_eps = 0.001,
    incremental_buffer=5,
    incremental_max_iter=1,
    incremental_grad_max_iter=2,
)

In [13]:
trainer.train(
    train_loader,
    test_loaders,
    optimizer,
    scheduler,
    regularizer=False,
    training_loss=train_loss,
    eval_losses=eval_losses,
)

Training on 100 samples
Testing on [100] samples         on resolutions [128].
Raw outputs of shape torch.Size([16, 1, 64, 64])
[0] time=10.79, avg_loss=0.9615, train_err=13.7352
Eval: 128_h1=0.8907, 128_l2=0.9276
[1] time=13.43, avg_loss=0.8565, train_err=12.2352
Eval: 128_h1=0.8324, 128_l2=0.6106
[2] time=11.76, avg_loss=0.8212, train_err=11.7312
Eval: 128_h1=0.8338, 128_l2=0.6176
[3] time=12.08, avg_loss=0.8158, train_err=11.6540
Eval: 128_h1=0.8334, 128_l2=0.5761
[4] time=11.67, avg_loss=0.8091, train_err=11.5589
Eval: 128_h1=0.8201, 128_l2=0.5892
[5] time=11.57, avg_loss=0.7925, train_err=11.3212
Eval: 128_h1=0.8149, 128_l2=0.5477
[6] time=11.00, avg_loss=0.7703, train_err=11.0045
Eval: 128_h1=0.8059, 128_l2=0.5359
[7] time=11.54, avg_loss=0.7463, train_err=10.6608
Eval: 128_h1=0.8044, 128_l2=0.5434
[8] time=10.85, avg_loss=0.7191, train_err=10.2728
Eval: 128_h1=0.7976, 128_l2=0.5091
[9] time=11.06, avg_loss=0.6938, train_err=9.9117
Eval: 128_h1=0.8103, 128_l2=0.5176


{'train_err': 9.911735875265938,
 'avg_loss': 0.6938215112686157,
 'avg_lasso_loss': None,
 'epoch_train_time': 11.058373041996674,
 '128_h1': tensor(0.8103),
 '128_l2': tensor(0.5176)}

In [14]:
# FNO
{'train_err': 9.714059012276786,
 'avg_loss': 0.679984130859375,
 'avg_lasso_loss': None,
 'epoch_train_time': 10.51430504200107,
 '128_h1': tensor(0.7340),
 '128_l2': tensor(0.5201)}

NameError: name 'tensor' is not defined

In [15]:
# HNO


In [16]:
# LNO


In [17]:
# WNO
